In [1]:
import warnings
warnings.filterwarnings("ignore")

import time
import os
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import seaborn as sns

import torch
import torch.nn as nn
import pytorch_lightning
from torch.utils.data import DataLoader
import torchvision.transforms as tt
import torch.optim as optim
from torch.optim import lr_scheduler
from sklearn.metrics import *
from PIL import Image
from imblearn.over_sampling import RandomOverSampler
from sklearn.utils.class_weight import compute_class_weight

from src.utils.utils import training, testing, EarlyStopping, get_y_true_preds
from src.utils.LungDataset import LungSet
from src.model.LungNetwork import LungNet

import torch

if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"Number of available GPUs: {num_gpus}")
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
seed=2024
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
device

Number of available GPUs: 1


device(type='cuda', index=0)

In [2]:
# model config
baseline_path = "src/model/tenpercent_resnet18.ckpt"
BATCH_SIZE = 24
num_classes = 6
prefix = '6C'

# data config
augmented_test = False
oversampling = False
train_path = f"data/splits/{prefix}_support_augmented.csv"
test_path = f"data/splits/{prefix}_query.csv" if not augmented_test else f"data/split/{prefix}_query_augmented.csv"
train_data_path = "data/augmented_data"
test_data_path = "data/normalized_data" if not augmented_test else "data/augmented_data"


save_dir = f"checkpoints/supervised/{prefix}/" if not oversampling else f"checkpoints/supervised/{prefix}/oversampled/"
save_path_model_best = f"{save_dir}/best_model.pth"
save_path_optim_best = f"{save_dir}/best_optim.pth"
save_path_model_last = f"{save_dir}/last_model.pth"
save_path_optim_last = f"{save_dir}/last_optim.pth"

save_path_loss_history = f"{save_dir}/train_test_loss_history.npy"
save_path_acc_history = f"{save_dir}/train_test_acc_history.npy"


In [3]:
dtrain= pd.read_csv(train_path).sample(frac = 1)
dtest = pd.read_csv(test_path).sample(frac = 1)

X_train = dtrain
y_train = dtrain['label']

# Oversample using sampling with replacement s.t. each class has the same number of samples
X_ros, y_ros = RandomOverSampler(random_state=seed).fit_resample(X_train, y_train)

In [4]:
X_train

,Patient,lame,patch,classe,tetraClass,label,dataset
67621,169,Fo,Fo_169_A (342)_VF.jpg,Foetal,T,5,train
28137,230,N,N_230_A (3)_rot270.jpg,Nécrose,Né,2,train
51341,231,S,S_231_A (1715)_VF.jpg,Solide,T,5,train
32559,9,N,N_9_S (1093).tif,Nécrose,Né,2,train
66549,230,Gc,Gc_230_A (145).jpg,Glandulaire complexe,T,5,train
...,...,...,...,...,...,...,...
47643,231,F,F_231_A (152)_rot45.jpg,Fibrose,Fi,4,train
35456,231,TL,TL_231_A (22)_HF.jpg,Tissu lymphoïde,TL,3,train
51808,7,A,A_7_A (279).jpg,Acinaire,T,5,train
52730,247,A,A_247_A (2937).jpg,Acinaire,T,5,train


In [5]:
X_train['tetraClass'].value_counts(), X_ros['tetraClass'].value_counts()

(tetraClass
 P     22755
 T     21165
 Fi    12612
 Né     8994
 TL     3069
 H      3006
 Name: count, dtype: int64,
 tetraClass
 T     22755
 Né    22755
 Fi    22755
 H     22755
 TL    22755
 P     22755
 Name: count, dtype: int64)

In [6]:
if oversampling:
    X_train, y_train = X_ros, y_ros

t_train = tt.Compose([tt.RandomHorizontalFlip(), 
                    tt.RandomVerticalFlip(),
                    tt.ToTensor()])
t_test = tt.Compose([tt.ToTensor()])

In [7]:
# create a LungSet object(torch.Datatset) with:
# df(DataFrame): image_path, label
# data_path(str): directory containing the images
# transform(transforms.Compose): transformations to apply
# __iter__: return torch(Image), label
train_set = LungSet(X_train, data_path=train_data_path)
test_set = LungSet(dtest, data_path=test_data_path)

# Create the torch.Dataloader
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

In [8]:
len(train_set), len(test_set)

(71601, 2623)

In [9]:
# Set up the model
model = LungNet(baseline_path, num_classes)
model = model.to(device)

# data paralellism in traning
# model=nn.DataParallel(model, device_ids=[0, 1])
# model=model.cuda()

criterion = nn.CrossEntropyLoss(label_smoothing=0.125)
early_stopping = EarlyStopping(patience=10, delta=0.001)
optimizer = optim.AdamW(model.parameters(), lr=0.1, weight_decay=0.01)
# allow dynamic lr reducing based on some measurement (a metric has stopped improving)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',factor=0.1, patience=10, min_lr=1e-15, verbose=True)

In [10]:
print("trainable parameters",sum([p.numel() for p in model.parameters() if p.requires_grad]))

trainable parameters 70310


In [11]:
best_acc = 0
num_epoch=50
comment=" "
T=0

train_loss, test_loss, train_accuracy, test_accuracy = [], [], [], []

for epoch in range(num_epoch):
    t0 = time.time()               
    # train one epoch for each batch                                                                                                         
    epoch_train_loss, epoch_train_accuracy  = training(model, train_loader, criterion, optimizer, device)
    epoch_test_loss, epoch_test_accuracy    = testing(model, test_loader, criterion, device)

    # save the model if is currently the best
    if best_acc < epoch_test_accuracy:
        best_acc = epoch_test_accuracy
        torch.save(model.state_dict(), save_path_model_best)
        torch.save(optimizer.state_dict(), save_path_optim_best)
        comment=" (loss decreased!) new best model saved"
    
    # reduce the learning rate if test loss didn't decrease
    scheduler.step(epoch_test_loss)

    # keep track of the loss and accuracy
    train_loss.append(epoch_train_loss)
    train_accuracy.append(epoch_train_accuracy)
    test_loss.append(epoch_test_loss)
    test_accuracy.append(epoch_test_accuracy)

    # every 5 epoch save the loss and accuracy history. the model and the optimizer (lr changes)
    if epoch!=0 and epoch%5 == 0:
        np.save(save_path_loss_history, [train_loss, test_loss])
        np.save(save_path_acc_history, [train_accuracy, test_accuracy])
        torch.save(model.state_dict(), save_path_model_last)
        torch.save(optimizer.state_dict(), save_path_optim_last)
        
    print('Epoch {:g}/{:g}: TRAIN Loss={:0.5f} -- Acc={:0.3f}% '.format(epoch+1,num_epoch, epoch_train_loss, epoch_train_accuracy), end='')
    print('|| TEST Loss={:0.5f} -- Acc={:0.3f}%  --- Time={:g}min'.format(epoch_test_loss,epoch_test_accuracy, (time.time()-t0)//60), end='')
    print(comment)
    comment=" "
    T+=(time.time()-t0)

    # early_stopping(epoch_test_loss)
    # if early_stopping.early_stop:
    #     print("Early stopping.")
    #     break
print("total time : T={:.3f}h".format(T//3600))

# save the final model
np.save(save_path_loss_history, [train_loss, test_loss])
np.save(save_path_acc_history, [train_accuracy, test_accuracy])
torch.save(model.state_dict(), save_path_model_last)
torch.save(optimizer.state_dict(), save_path_optim_last)

100%|██████████| 2984/2984 [21:14<00:00,  2.34it/s]


Epoch 1/50: TRAIN Loss=0.73201 -- Acc=91.110% || TEST Loss=6.32105 -- Acc=42.699%  --- Time=22min (loss decreased!) new best model saved


100%|██████████| 2984/2984 [21:24<00:00,  2.32it/s]


Epoch 2/50: TRAIN Loss=0.70438 -- Acc=92.454% || TEST Loss=0.88435 -- Acc=82.920%  --- Time=22min (loss decreased!) new best model saved


100%|██████████| 2984/2984 [21:28<00:00,  2.32it/s]


...

Epoch 48/50: TRAIN Loss=0.61357 -- Acc=95.941% || TEST Loss=0.78895 -- Acc=87.038%  --- Time=22min 


100%|██████████| 2984/2984 [21:54<00:00,  2.27it/s]


Epoch 49/50: TRAIN Loss=0.61281 -- Acc=95.993% || TEST Loss=0.83060 -- Acc=85.551%  --- Time=22min 


100%|██████████| 2984/2984 [21:55<00:00,  2.27it/s]


Epoch 50/50: TRAIN Loss=0.61196 -- Acc=96.073% || TEST Loss=0.80750 -- Acc=86.237%  --- Time=22min 
total time : T=18.000h
